## Comenzamos con la importancion de las librerias necesarias para este proyecto

In [1]:
import numpy as np
import pandas as pd
import pyarrow
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

## Carga de los archivos CSV

In [2]:
# Lectura de los archivos originales .csv provistos.
data_movies = pd.read_csv('../datasets_originales/Movies/movies_dataset.csv', low_memory=False)
data_credits = pd.read_csv('../datasets_originales/Movies/credits.csv', low_memory=False)

## Comenzamos con el ETL para el datasets data_movies

* Reemplazo de valores nulero de los campos revenue por 0
* Reemplazo de valores nulero de los campos budget por 0
* Eliminacion de los campos nulos de release_date
* Cambiar formato AAAA-mm-dd de la columna release_date
* Creacion de la columna release_year

In [3]:
# Conversion de las columnas revenue, budget numericos
# Intercambio nulos por Ceros
data_movies['revenue'] = pd.to_numeric(data_movies['revenue'], errors='coerce').fillna(0)
data_movies['budget'] = pd.to_numeric(data_movies['budget'], errors='coerce').fillna(0)

# Eliminacion de nulos de la columna release_date y
# paso todo al tipo de dato date y el formato YYYY-mm-dd
data_movies = data_movies.dropna(subset='release_date')
data_movies = data_movies[data_movies['release_date'] != '0']
data_movies['release_date'] = pd.to_datetime(data_movies['release_date'], format='%Y-%m-%d', errors='coerce')

# Creando la columna release_year
data_movies['release_year'] = data_movies['release_date'].dt.year

# Creando la columna return
data_movies['return'] = data_movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

In [4]:
# Conversion la columna 'id' a numerico.
data_movies['id'] = pd.to_numeric(data_movies['id'], errors='coerce')

# Eliminando las filas que tengan NaN en la columna 'id'
data_movies = data_movies.dropna(subset=['id'])

# Convertir la columna 'id' a tipo entero
data_movies['id'] = data_movies['id'].astype(int)

In [19]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  45376 non-null  object        
 1   belongs_to_collection  4488 non-null   object        
 2   budget                 45376 non-null  float64       
 3   genres                 45376 non-null  object        
 4   homepage               7766 non-null   object        
 5   id                     45376 non-null  int64         
 6   imdb_id                45362 non-null  object        
 7   original_language      45365 non-null  object        
 8   original_title         45376 non-null  object        
 9   overview               44435 non-null  object        
 10  popularity             45376 non-null  object        
 11  poster_path            45037 non-null  object        
 12  production_companies   45376 non-null  object        
 13  produc

### Eliminacion de algunas columnas que no seran utilizadas

* video
* imdb
* adult
* original_title
* poster_path
* homepage
* tagline
* runtime
* belongs_to_collection

In [5]:
# Elimino las columnas que no se van a utilizar antes de seguir
# con el cambio de valores nulos o tipos de datos
columnas_eliminar = ['video','imdb_id','adult','original_title',
                     'tagline','runtime','poster_path','homepage',
                     'belongs_to_collection']
data_movies = data_movies.drop(columns=columnas_eliminar)

### Desanidando la columna genres: Los pasos los explicamos arriba de cada linea del codigo.

In [6]:
# Convertimos el texto de la columna 'genres' a una lista de diccionarios
data_movies['genres'] = data_movies['genres'].apply(ast.literal_eval)

# Desanidamos la lista de generos y creamos una fila por cada genero en una pelicula
data_movies = data_movies.explode('genres')

# Extraemos el nombre del genero de cada diccionario y lo guarda en una nueva columna 'genre_name'
data_movies['genre_name'] = data_movies['genres'].apply(lambda x: x['name'] if isinstance(x, dict) else None)

# Eliminamos la columna original 'genres'
data_movies = data_movies.drop(columns=['genres'])

### Teniendo en cuenta el modelo final (MVP) elimino otras columnas que luego no voy a tener en cuenta. 

* production_companies
* production_countries
* spoken_languages

In [7]:
columnas_eliminar = ['production_companies','production_countries',
                     'spoken_languages']
data_movies = data_movies.drop(columns=columnas_eliminar)

### Verificamos tipos de datos

In [23]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93420 entries, 0 to 45465
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             93420 non-null  float64       
 1   id                 93420 non-null  int64         
 2   original_language  93403 non-null  object        
 3   overview           92104 non-null  object        
 4   popularity         93420 non-null  object        
 5   release_date       93420 non-null  datetime64[ns]
 6   revenue            93420 non-null  float64       
 7   status             93296 non-null  object        
 8   title              93420 non-null  object        
 9   vote_average       93420 non-null  float64       
 10  vote_count         93420 non-null  float64       
 11  release_year       93420 non-null  float64       
 12  return             93420 non-null  float64       
 13  genre_name         91036 non-null  object        
dtypes: datetime

### Cambiando algunos tipos de datos

* Columna release_year
* Columna genre_name
* Columna title
* Columna overviwe

In [8]:
# Convirtiendo la columna 'release_year' a tipo datetime
data_movies['release_year'] = data_movies['release_year'].astype('datetime64[ns]')

# Convirtiendo la columna 'genre_name' a tipo string
data_movies['genre_name'] = data_movies['genre_name'].astype(str)

# Convirtiendo la columna 'title' a tipo string
data_movies['title'] = data_movies['title'].astype(str)

# Convirtiendo la columna 'overview' a tipo string
data_movies['overview'] = data_movies['overview'].astype(str)

In [34]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93420 entries, 0 to 45465
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             93420 non-null  float64       
 1   id                 93420 non-null  int64         
 2   original_language  93403 non-null  object        
 3   overview           93420 non-null  object        
 4   popularity         93420 non-null  object        
 5   release_date       93420 non-null  datetime64[ns]
 6   revenue            93420 non-null  float64       
 7   status             93296 non-null  object        
 8   title              93420 non-null  object        
 9   vote_average       93420 non-null  float64       
 10  vote_count         93420 non-null  float64       
 11  release_year       93420 non-null  datetime64[ns]
 12  return             93420 non-null  float64       
 13  genre_name         93420 non-null  object        
dtypes: datetime

In [35]:
data_movies.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'status', 'title', 'vote_average',
       'vote_count', 'release_year', 'return', 'genre_name'],
      dtype='object')

### Guardamos el archivo en parquet en la carpeta Source. De ahi sera consumido por la API.

In [9]:
data_movies.to_parquet('../Source/data_movies.parquet')

***
---

## ETL para el dataset data_credits

### Pasos a seguir:

* Desanidar la columna crew para quedarme con los id de peliculas y los nombres de los directores.
* Desanidar la columna cast para quedarme con los id de peliculas y los nombres de los actores.
* Seran 2 datasets diferentes data_credits_actores y data_credits_directores. Los cuales los voy a usar en conjunto con el data_movies.

### Desanidando la columna crew

In [10]:
# Conviertiendo las cadenas en la columna 'crew' a listas de diccionarios.
data_credits['crew'] = data_credits['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Desanidando la columna 'crew' (explode) creando una fila por cada elemento en la lista
data_credits_exploded = data_credits.explode('crew')

# Restablezco el indice por problemas con duplicados despues de desanidar
data_credits_exploded = data_credits_exploded.reset_index(drop=True)

# Normalizo los diccionarios dentro de la columna 'crew' en columnas separadas
data_credits_crew = pd.json_normalize(data_credits_exploded['crew'])

# Mantengo la columna 'id' de la pelicula en el dataset normalizado
data_credits_crew['id'] = data_credits_exploded['id']

# Filtrado de las filas que tengan 'Director'
data_credits_directores = data_credits_crew[data_credits_crew['job'] == 'Director']

# Me quedo solo con las columnas 'id' y 'name', y renombra la columna 'name' a 'director'
data_credits_directores = data_credits_directores[['id', 'name']].rename(columns={'name': 'director'})

In [30]:
data_credits_directores.head()

,id,director
0,862,John Lasseter
109,8844,Joe Johnston
122,15602,Howard Deutch
126,31357,Forest Whitaker
141,11862,Charles Shyer


### Desanidando la columna cast

In [11]:
# Convierto las cadenas en la columna 'cast' a listas de diccionarios.
data_credits['cast'] = data_credits['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Desanido la columna 'cast', creando una fila por cada elemento en la lista
data_credits_exploded = data_credits.explode('cast')

# Restablezco el indice por problemas con duplicados despues de desanidar
data_credits_exploded = data_credits_exploded.reset_index(drop=True)

# Normalizo los diccionarios dentro de la columna 'cast' en columnas separadas
data_credits_cast = pd.json_normalize(data_credits_exploded['cast'])

# Mantengo la columna 'id' de la pelicula en el dataset normalizado
data_credits_cast['id'] = data_credits_exploded['id']

# Me quedo solo con las columnas 'id' y 'name', y renombra la columna 'name' a 'actor'
data_credits_actores = data_credits_cast[['id', 'name']].rename(columns={'name': 'actor'})

In [14]:
data_credits_actores.head()

,id,actor
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles
3,862,Jim Varney
4,862,Wallace Shawn


### Guardamos ambos dataset en parque para luego ser consumidos por la API

In [12]:
data_credits_actores.to_parquet('../Source/data_credits_actores.parquet')
data_credits_directores.to_parquet('../Source/data_credits_directores.parquet')